In [1]:
!pip install python-telegram-bot wikipedia scikit-learn pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=e43b33d50cb6a450ca722c1a28b326250f42d41d6d4e4b1389039f6f1d79ab94
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [4]:
import logging
from telegram import Update, ForceReply
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ConversationHandler, CallbackContext
import wikipedia
import nest_asyncio
import asyncio

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

logger = logging.getLogger(__name__)

wikipedia.set_lang("pt")

CHOOSING, SYMPTOMS = range(2)


async def start(update: Update, context: CallbackContext) -> int:
    user = update.effective_user
    await update.message.reply_html(
        rf'Olá {user.mention_html()}! Eu sou um bot que pode ajudar com diagnósticos. Escolha uma categoria:'
        '\n1 - Dengue\n'
        '2 - Covid\n'
        '3 - Listar meus sintomas\n'
        'Digite /cancel para encerrar a conversa para qualquer conversa.',
        reply_markup=ForceReply(selective=True),
    )
    return CHOOSING

async def choose_category(update: Update, context: CallbackContext) -> int:
    user_text = update.message.text
    if user_text == '1':
        summary = wikipedia.summary('Dengue', sentences=2, auto_suggest=False)
        await update.message.reply_text(f'{summary}\n\nEscolha uma opção:\n3 - Listar meus sintomas\nDigite /cancel para encerrar a conversa a qualquer momento')
        return CHOOSING
    elif user_text == '2':
        summary = wikipedia.summary('COVID-19', sentences=2, auto_suggest=False)
        await update.message.reply_text(f'{summary}\n\nEscolha uma opção:\n3 - Listar meus sintomas\nDigite /cancel para encerrar a conversa a qualquer momento')
        return CHOOSING
    elif user_text == '3':
        await update.message.reply_text('Por favor, liste seus sintomas, separados por "-".')
        return SYMPTOMS
    else:
        await update.message.reply_text('Escolha inválida. Por favor, escolha novamente:\n1 - Dengue\n2 - Covid\n3 - Listar meus sintomas')
        return CHOOSING

async def list_symptoms(update: Update, context: CallbackContext) -> int:
    user_symptoms = update.message.text.lower().split('-')
    user_symptoms = {symptom.strip() for symptom in user_symptoms}

    dengue_symptoms = {'febre alta', 'dor de cabeça', 'dor atrás dos olhos', 'dor nas articulações', 'erupção cutânea'}
    covid_symptoms = {'febre', 'tosse', 'falta de ar', 'perda de paladar', 'perda de olfato'}

    dengue_count = len(dengue_symptoms.intersection(user_symptoms))
    covid_count = len(covid_symptoms.intersection(user_symptoms))

    if dengue_count > covid_count:
        await update.message.reply_text('Baseado nos seus sintomas, é mais provável que seja Dengue.\nPor favor não se apoie apenas nestes diagnósticos, sempre consulte um médico para maior confirmação.\n\nEscolha outra opção: \n1 - Dengue\n2 - Covid\n3 - Listar meus sintomas.\nDigite /cancel para encerrar a conversa.')
    elif covid_count > dengue_count:
        await update.message.reply_text('Baseado nos seus sintomas, é mais provável que seja COVID-19.\nPor favor não se apoie apenas nestes diagnósticos, sempre consulte um médico para maior confirmação.\n\nEscolha outra opção: \n1 - Dengue\n2 - Covid\n3 - Listar meus sintomas.\nDigite /cancel para encerrar a conversa.')
    else:
        await update.message.reply_text('Não é possível determinar com clareza a partir desta lisatgem. Consulte um médico para melhor compreensão.')

    return ConversationHandler.END

async def cancel(update: Update, context: CallbackContext) -> int:
    user = update.effective_user
    await update.message.reply_html(
        rf'Adeus {user.mention_html()}! Espero ter ajudado.',
    )
    return ConversationHandler.END

def main() -> None:
    nest_asyncio.apply()

    application = Application.builder().token("7102326706:AAG3HsnTjzCNVClOSE9QpeDPz_IwDCtxjAo").build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [MessageHandler(filters.TEXT & ~filters.COMMAND, choose_category)],
            SYMPTOMS: [MessageHandler(filters.TEXT & ~filters.COMMAND, list_symptoms)],
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    application.add_handler(conv_handler)
    application.run_polling()


In [ ]:
if __name__ == '__main__':
    asyncio.run(main())